[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/dreamgaussian-colab/blob/main/dreamgaussian_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/dreamgaussian

!pip install -q torch-ema einops tensorboardX plyfile dearpygui huggingface_hub diffusers accelerate transformers xatlas 
!pip install -q trimesh PyMCubes pymeshlab rembg[gpu,cli] omegaconf ninja gradio
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.1.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/simple_knn-0.0.0-cp310-cp310-linux_x86_64.1.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/nvdiffrast-0.3.1-py3-none-any.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/kiui-0.1.8-py3-none-any.whl
!apt install -qq -y xvfb

%cd /content/dreamgaussian

import re
import subprocess
import gradio as gr

def create_from_text(prompt):
    sanitized_prompt = re.sub('[^0-9a-zA-Z]+', '_', prompt)
    cmd1 = ["python", "main.py", "--config", "configs/text.yaml", f"prompt={prompt}", f"save_path={sanitized_prompt}"]
    cmd2 = ["python", "main2.py", "--config", "configs/text.yaml", f"prompt={prompt}", f"save_path={sanitized_prompt}"]
    cmd3 = ["xvfb-run", "python", "-m", "kiui.render", f"logs/{sanitized_prompt}.obj", "--save_video", f"{sanitized_prompt}.mp4"]
    subprocess.run(cmd1)
    subprocess.run(cmd2)
    subprocess.run(cmd3)
    return f"{sanitized_prompt}.mp4"

def create_from_image(image):
    sanitized_prompt = 'image'
    image.save(f"{sanitized_prompt}.png")
    cmd1 = ["python", "process.py", f"{sanitized_prompt}.png", "--size", "512"] # <<< 256 may be a good default
    cmd2 = ["python", "main.py", "--config", "configs/image.yaml", f"input={sanitized_prompt}_rgba.png", f"save_path={sanitized_prompt}"]
    cmd3 = ["python", "main2.py", "--config", "configs/image.yaml", f"input={sanitized_prompt}_rgba.png", f"save_path={sanitized_prompt}"]
    cmd4 = ["xvfb-run", "python", "-m", "kiui.render", f"logs/{sanitized_prompt}.obj", "--save_video", f"{sanitized_prompt}.mp4"]
    subprocess.run(cmd1)
    subprocess.run(cmd2)
    subprocess.run(cmd3)
    subprocess.run(cmd4)
    return f"{sanitized_prompt}.mp4"

with gr.Blocks() as demo:
    prompt = gr.Textbox(lines=2, placeholder="Enter Prompt...")
    image = gr.Image(type="pil", label="Image")
    btn = gr.Button("Generate from text")
    btn_2 = gr.Button("Generate from image")
    video = gr.Video(type="mp4")
    btn.click(create_from_text, inputs=prompt, outputs=video)
    btn_2.click(create_from_image, inputs=image, outputs=video)
    
demo.queue().launch(share=True, inline=False, debug=True)